# 在 PyTorch 中使用 TensorBoard
## 开始之前

要运行本教程，您需要安装 PyTorch、TorchVision、Matplotlib 和 TensorBoard。

使用 `conda`：

`conda install pytorch torchvision -c pytorch`
`conda install matplotlib tensorboard`

使用 `pip`：

`pip install torch torchvision matplotlib tensorboard`

安装依赖项后，请在安装它们的 Python 环境中重新启动此笔记本。

## 简介

在本笔记本中，我们将针对 Fashion-MNIST 数据集训练 LeNet-5 的一个变体。Fashion-MNIST 是一组图像块，描绘了各种服装，并带有十个类别标签，指示所描绘的服装类型。

In [1]:
# PyTorch 模型和训练所需的库
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# 图像数据集和图像处理
import torchvision
import torchvision.transforms as transforms

# 图像显示
import matplotlib.pyplot as plt
import numpy as np

# PyTorch TensorBoard 支持
from torch.utils.tensorboard import SummaryWriter

ModuleNotFoundError: No module named 'tensorboard'

## 在 TensorBoard 中显示图像

让我们从将数据集中的示例图像添加到 TensorBoard 开始：

In [ ]:
# 加载数据集并准备使用
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))])

# 将训练集和验证集分别存储在 ./data 中
training_set = torchvision.datasets.FashionMNIST('./data',
    download=True,
    train=True,
    transform=transform)
validation_set = torchvision.datasets.FashionMNIST('./data',
    download=True,
    train=False,
    transform=transform)

training_loader = torch.utils.data.DataLoader(training_set,
                                              batch_size=4,
                                              shuffle=True,
                                              num_workers=2)

validation_loader = torch.utils.data.DataLoader(validation_set,
                                                batch_size=4,
                                                shuffle=False,
                                                num_workers=2)

# 类别标签
classes = ('T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
        'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle Boot')

In [ ]:
# 辅助函数，用于内联显示图像
def matplotlib_imshow(img, one_channel=False):
    if one_channel:
        img = img.mean(dim=0)
    img = img / 2 + 0.5     # 反归一化
    npimg = img.numpy()
    if one_channel:
        plt.imshow(npimg, cmap="Greys")
    else:
        plt.imshow(np.transpose(npimg, (1, 2, 0)))

In [ ]:
# 提取一个包含 4 张图像的批次
dataiter = iter(training_loader)
images, labels = dataiter.next()

# 从图像创建网格并显示它们
img_grid = torchvision.utils.make_grid(images)
matplotlib_imshow(img_grid, one_channel=True)

上面，我们使用 TorchVision 和 Matplotlib 创建了一个输入数据小批量的可视化网格。下面，我们使用 `SummaryWriter` 的 `add_image()` 方法将图像记录到 TensorBoard，并调用 `flush()` 确保其立即写入磁盘。

In [ ]:
# 默认 log_dir 参数是 "runs" - 但最好具体一些
# torch.utils.tensorboard.SummaryWriter 在上面已导入
writer = SummaryWriter('runs/fashion_mnist_experiment_1')

# 将图像数据写入 TensorBoard 日志目录
writer.add_image('Four Fashion-MNIST Images', img_grid)
writer.flush()

# 要查看，请在命令行启动 TensorBoard：
#   tensorboard --logdir=runs
# ...然后打开浏览器标签页，访问 http://localhost:6006/

如果您在命令行启动 TensorBoard 并在新浏览器标签页中打开它（通常位于 [localhost:6006](localhost:6006)），您应该可以在 IMAGES 标签下看到图像网格。

## 绘制标量以可视化训练

TensorBoard 对于跟踪训练的进展和效果非常有用。下面，我们将运行一个训练循环，跟踪一些指标，并保存数据以供 TensorBoard 使用。

让我们定义一个模型来分类我们的图像块，以及一个优化器和损失函数来进行训练：

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    

net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

现在让我们训练一个 epoch，并每 1000 个批次评估一次训练集和验证集的损失：

In [ ]:
print(len(validation_loader))
for epoch in range(1):  # 数据集循环多次
    running_loss = 0.0

    for i, data in enumerate(training_loader, 0):
        # 基本训练循环
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 1000 == 999:    # 每 1000 个小批次...
            print('批次 {}'.format(i + 1))
            # 检查验证集
            running_vloss = 0.0
            
            net.train(False) # 验证时不需要跟踪梯度
            for j, vdata in enumerate(validation_loader, 0):
                vinputs, vlabels = vdata
                voutputs = net(vinputs)
                vloss = criterion(voutputs, vlabels)
                running_vloss += vloss.item()
            net.train(True) # 训练时重新打开梯度
            
            avg_loss = running_loss / 1000
            avg_vloss = running_vloss / len(validation_loader)
            
            # 记录每批次的平均损失
            writer.add_scalars('训练 vs 验证损失',
                            { '训练' : avg_loss, '验证' : avg_vloss },
                            epoch * len(training_loader) + i)

            running_loss = 0.0
print('训练完成')

writer.flush()

切换到打开的 TensorBoard，查看 SCALARS 标签。

## 可视化您的模型
TensorBoard 还可以用于检查模型中的数据流。为此，请使用模型和示例输入调用 `add_graph()` 方法。当您打开

In [ ]:
# 再次获取一个小批量图像
dataiter = iter(training_loader)
images, labels = dataiter.next()

# add_graph() 将通过模型跟踪示例输入，
# 并将其呈现为图形。
writer.add_graph(net, images)
writer.flush()

当您切换到 TensorBoard 时，您应该会看到一个 GRAPHS 标签。双击 "NET" 节点以查看模型中的层和数据流。

## 使用嵌入可视化数据集

我们使用的 28x28 图像块可以建模为 784 维向量（28 * 28 = 784）。将其投影到低维表示可能会很有启发性。`add_embedding()` 方法会自动将数据投影到方差最高的三个维度，并将其显示为交互式 3D 图表。

下面，我们将从数据中取一个样本，并生成这样的嵌入：

In [ ]:
# 选择随机子集数据及其对应标签
def select_n_random(data, labels, n=100):
    assert len(data) == len(labels)

    perm = torch.randperm(len(data))
    return data[perm][:n], labels[perm][:n]

# 提取随机子集数据
images, labels = select_n_random(training_set.data, training_set.targets)

# 获取每张图像的类别标签
class_labels = [classes[label] for label in labels]

# 记录嵌入
features = images.view(-1, 28 * 28)
writer.add_embedding(features,
                    metadata=class_labels,
                    label_img=images.unsqueeze(1))
writer.flush()
writer.close()

现在，如果您切换到 TensorBoard 并选择 PROJECTOR 标签，您应该会看到投影的 3D 表示。您可以旋转和缩放模型。在大尺度和小尺度上检查它，并查看是否可以在投影数据和标签的聚类中发现模式。

**注意：** 为了更好地可视化，建议：
* 从左侧的 "Color by" 下拉菜单中选择 "label"
* 切换顶部的夜间模式图标，将浅色图像放在深色背景上。

## 其他资源

有关更多信息，请参阅：
* [PyTorch 文档中关于 `torch.utils.tensorboard.SummaryWriter`](https://pytorch.org/docs/stable/tensorboard.html?highlight=summarywriter) 的内容
* [PyTorch.org 教程](https://pytorch.org/tutorials/) 中的 TensorBoard 教程内容
* 有关 TensorBoard 的更多信息，请参阅 [TensorBoard 文档](https://www.tensorflow.org/tensorboard)